# RNN을 이용한 텍스트 생성

In [2]:
import numpy as np
import tensorflow as tf
seed = 2022
np.random.seed(seed)
tf.random.set_seed(seed)

In [3]:
text = '''경마장에 있는 말이 뛰고 있다.
그의 말이 법이다.
가는 말이 고와야 오는 말이 곱다.'''

## 1.텍스트전처리
- keras와 sklearn이 다름

In [4]:
from tensorflow.keras.preprocessing.text import Tokenizer           # X
from tensorflow.keras.preprocessing.sequence import pad_sequences   # X
from tensorflow.keras.utils import to_categorical                   # y

### 1.1.단어 집합 생성

In [5]:
token = Tokenizer()
token.fit_on_texts([text])        # 머신러닝에서의 fit_transform

In [6]:
# BOW 같은 효과
#   특징
#    1) 가나다 순서가 아닌, 많이 나온 순서로 되어있음
#    2) index가 1부터 메겨짐. (timesteps라는 제약 때운)
#       어떤 데이터는 채우고 어떤 데이터는 자름.
#       채우는 것을 padding이라고 하고, 보통 0으로 채움
#       padding 시 0이 사용되기 때문에 단어 매핑이 1부터 시작함
token.word_index

{'가는': 8,
 '경마장에': 2,
 '고와야': 9,
 '곱다': 11,
 '그의': 6,
 '뛰고': 4,
 '말이': 1,
 '법이다': 7,
 '오는': 10,
 '있는': 3,
 '있다': 5}

In [7]:
# 단어 집합 크기
vocab_size = len(token.word_index) + 1      # 0때문에 1을 더해줘야 됨.
vocab_size

12

In [8]:
token.texts_to_sequences(['그의 말이 법이다.'])

[[6, 1, 7]]

In [9]:
# Corpus를 인코딩한 결과
for line in text.split('\n'):
    encoded = token.texts_to_sequences([line])[0]  # 항상 리스트로 감쌀 것
    print(encoded)

[2, 3, 1, 4, 5]
[6, 1, 7]
[8, 1, 9, 10, 1, 11]


## 2.Sequence 생성

In [12]:
sequences = []
for line in text.split('\n'):
    encoded = token.texts_to_sequences([line])[0] 
    for i in range(1, len(encoded)):
        sequences.append(encoded[:i+1])
sequences

[[2, 3],
 [2, 3, 1],
 [2, 3, 1, 4],
 [2, 3, 1, 4, 5],
 [6, 1],
 [6, 1, 7],
 [8, 1],
 [8, 1, 9],
 [8, 1, 9, 10],
 [8, 1, 9, 10, 1],
 [8, 1, 9, 10, 1, 11]]

In [15]:
# sequence 중 가장 긴 것
max_len = max(len(seq) for seq in sequences)
max_len

6

### 2.1. Padding
- 전체 샘플 길이를 max_len에 맞추도록 0을 추가
- 앞 부분에 0을 채울것
- 앞 쪽을 채우는 pre 옵션이 default 값임

In [16]:
sequences = pad_sequences(sequences, maxlen = max_len, padding = 'pre') # padding = 'pre'는 안써줘도 됨
sequences

array([[ 0,  0,  0,  0,  2,  3],
       [ 0,  0,  0,  2,  3,  1],
       [ 0,  0,  2,  3,  1,  4],
       [ 0,  2,  3,  1,  4,  5],
       [ 0,  0,  0,  0,  6,  1],
       [ 0,  0,  0,  6,  1,  7],
       [ 0,  0,  0,  0,  8,  1],
       [ 0,  0,  0,  8,  1,  9],
       [ 0,  0,  8,  1,  9, 10],
       [ 0,  8,  1,  9, 10,  1],
       [ 8,  1,  9, 10,  1, 11]], dtype=int32)

In [23]:
X = sequences[:, :-1]
y = sequences[:, -1]
X.shape, y.shape

((11, 5), (11,))

In [24]:
Y = to_categorical(y)
Y.shape

(11, 12)

## 3.모델
1. Embedding : 밀집벡터 형태로 바꿈
     - 문장의 개수(3) * 단어의 수(11)에서 11보다 차원을 줄임
     - 노드 수 의 고민이 사라짐?
2. SimpleRNN
3. Dense

In [26]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense 

In [27]:
# Embedding = 4, SimpleRNN = 32
model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=4, input_length = max_len - 1),
    SimpleRNN(units = 32),
    Dense(vocab_size, activation = 'softmax')
])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 5, 4)              48        
                                                                 
 simple_rnn (SimpleRNN)      (None, 32)                1184      
                                                                 
 dense (Dense)               (None, 12)                396       
                                                                 
Total params: 1,628
Trainable params: 1,628
Non-trainable params: 0
_________________________________________________________________


In [44]:
(4  + 32) * 32 + 32
# Embedding의 개수 4가 RNN 입장에서는 feature 개수가 됨(input)dim
# 그렇게 때문에 RNN에서는 이제 unit의 수만 써주면 됨

1184

In [28]:
model.compile('adam', 'categorical_crossentropy', metrics=['accuracy'])

In [29]:
hist = model.fit(X, Y, epochs = 200, verbose = 0)

In [31]:
hist.history['accuracy'][-1]

1.0

### 3.1.모델검증

In [34]:
from my_util import sentence_generation
sentence_generation(model, token, max_len, '경마장에', 5)
# 모델, 토크나이저, maxlenm 현재 단어, 반복횟수

'경마장에 있는 말이 뛰고 있다 곱다'

In [35]:
sentence_generation(model, token, max_len, '가는', 5)

'가는 말이 고와야 오는 말이 곱다'

In [39]:
sentence_generation(model, token, max_len, '오는', 2)

'오는 말이 말이'

## 4.모델변화
- Embedding vector 개수 [2, 4, 6]
- RNN 유닛 개수 [24, 32, 48]

In [42]:
for n_embed in [2, 4, 6]:
    for n_unit in [24, 32, 48]:
        model = Sequential([
            Embedding(input_dim = vocab_size, output_dim = n_embed, input_length = max_len - 1),
            SimpleRNN(units = n_unit),
            Dense(vocab_size, activation = 'softmax')
        ])
        model.compile('adam', 'categorical_crossentropy', metrics=['accuracy'])
        hist = model.fit(X, Y, epochs = 200, verbose = 0)
        print('='*20)
        print(f'Embedding vector :{n_embed}, RNN_unit :{n_unit}')
        print(f'정확도{hist.history["accuracy"][-1]:.4f}')
        print(sentence_generation(model, token, max_len, '경마장에', 4))
        print(sentence_generation(model, token, max_len, '그의', 2))
        print(sentence_generation(model, token, max_len, '가는', 5))
        print()

Embedding vector :2, RNN_unit :24
정확도0.9091
경마장에 말이 말이 뛰고 있다
그의 말이 법이다
가는 말이 고와야 오는 말이 곱다

Embedding vector :2, RNN_unit :32
정확도0.8182
경마장에 말이 말이 뛰고 있다
그의 말이 법이다
가는 말이 말이 말이 말이 곱다

Embedding vector :2, RNN_unit :48
정확도1.0000
경마장에 있는 말이 뛰고 있다
그의 말이 법이다
가는 말이 고와야 오는 말이 곱다

Embedding vector :4, RNN_unit :24
정확도0.9091
경마장에 말이 말이 뛰고 있다
그의 말이 법이다
가는 말이 고와야 오는 말이 곱다

Embedding vector :4, RNN_unit :32
정확도0.9091
경마장에 말이 법이다 오는 말이
그의 말이 법이다
가는 말이 고와야 오는 말이 곱다

Embedding vector :4, RNN_unit :48
정확도0.9091
경마장에 말이 말이 뛰고 있다
그의 말이 법이다
가는 말이 고와야 오는 말이 곱다

Embedding vector :6, RNN_unit :24
정확도1.0000
경마장에 있는 말이 뛰고 있다
그의 말이 법이다
가는 말이 고와야 오는 말이 곱다

Embedding vector :6, RNN_unit :32
정확도1.0000
경마장에 있는 말이 뛰고 있다
그의 말이 법이다
가는 말이 고와야 오는 말이 곱다

Embedding vector :6, RNN_unit :48
정확도1.0000
경마장에 있는 말이 뛰고 있다
그의 말이 법이다
가는 말이 고와야 오는 말이 곱다

